In [ ]:
import torch
import torch.nn as nn
import os

# --- [필수] 1. 'external/time-llm'에서 PatchTST 모델 임포트 ---
# (노트북 표준 세팅이 선행되어야 함)
try:
    # 'external/time-llm'이 sys.path에 있으므로,
    # models 폴더의 PatchTST.py에서 Model 클래스를 가져옴
    from models.PatchTST import Model as PatchTST
    
    print("Successfully imported PatchTST model from external submodule.")
    
except ImportError:
    print("[!!!] Error: Could not import 'models.PatchTST.Model'")
    print("Please check if:")
    print("  1. 'external/time-llm' submodule is initialized (git submodule update --init)")
    print("  2. The 'Standard Code Setting' snippet (setting sys.path) was executed.")
except Exception as e:
    print(f"An unexpected error occurred during import: {e}")

# --- 2. 모델 하이퍼파라미터 정의 ---

# [A] 데이터 차원 (Phase 2에서 정의한 값)
INPUT_SEQ_LEN = 120  # 입력 시퀀스 길이 (120일)
OUTPUT_SEQ_LEN = 10  # 예측 시퀀스 길이 (10일)
N_CHANNELS = 12      # 입력/출력 채널 수 (12개)

# [B] PatchTST 모델 파라미터 (주요 튜닝 대상)
PATCH_LEN = 16       # 120일치 데이터를 16일짜리 '조각(Patch)'으로 나눔
STRIDE = 8           # 조각을 8일씩 겹치면서 나눔
D_MODEL = 128        # 모델의 주요 은닉층(Hidden Layer) 크기
N_HEADS = 8          # 어텐션(Attention) 헤드 수
N_LAYERS = 3         # PatchTST 인코더 레이어 수
D_FF = 256           # 피드포워드 네트워크 크기
DROPOUT = 0.1

# --- 3. 모델 설정(Configuration) 및 초기화 ---

# 3-1. 디바이스 설정 (GPU 우선)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 3-2. PatchTST 모델 설정값(config) 정의
model_config = {
    'c_in': N_CHANNELS,
    'context_window': INPUT_SEQ_LEN,
    'target_window': OUTPUT_SEQ_LEN,
    
    'patch_len': PATCH_LEN,
    'stride': STRIDE,
    'd_model': D_MODEL,
    'n_heads': N_HEADS,
    'd_ff': D_FF,
    'num_layers': N_LAYERS,
    'dropout': DROPOUT,
    
    # [중요] PatchTST 내부의 헤드(Linear)가 10일치(target_window) 예측을
    # 12채널(c_in) 모두에 대해 수행하도록 설정
    'pred_len': OUTPUT_SEQ_LEN, 
    'output_dim': N_CHANNELS, 
    
    'device': device
}

# 3-3. 모델 초기화 (Instantiate)
try:
    if 'PatchTST' in locals():
        model = PatchTST(**model_config).to(device)
        print("\n[SUCCESS] PatchTST model initialized successfully.")
    else:
        print("\n[FAIL] PatchTST class not found. Check import.")

except Exception as e:
    print(f"\n[FAIL] Error initializing model: {e}")


# --- 4. [검증] 모델 입/출력 Shape 테스트 ---
# (모델이 성공적으로 초기화되었을 경우)
if 'model' in locals():
    
    # 4-1. 가상의 3D 텐서 (배치=32개) 생성
    # (Batch Size, Input Seq Len, Channels)
    dummy_x = torch.randn(32, INPUT_SEQ_LEN, N_CHANNELS).to(device)
    
    # 4-2. 모델 순전파 (Forward Pass)
    # (모델은 120일치 입력을 받아 10일치 예측을 반환해야 함)
    model.eval() # (추론 모드로 설정)
    with torch.no_grad():
        y_pred = model(dummy_x)
    
    # 4-3. Shape 확인
    print("\n--- Model I/O Shape Test ---")
    print(f"Input Shape (X): {dummy_x.shape}")
    print(f"Output Shape (y_pred): {y_pred.shape}")
    
    # [검증] 출력 Shape이 (Batch Size, Output Seq Len, Channels)인지 확인
    expected_shape = (32, OUTPUT_SEQ_LEN, N_CHANNELS)
    if y_pred.shape == expected_shape:
        print(f"✅ [SUCCESS] Output shape {y_pred.shape} matches expected {expected_shape}")
    else:
        print(f"❌ [FAIL] Output shape is {y_pred.shape}, but expected {expected_shape}")

: 